In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
df = pd.read_csv('infy(97-98).csv')

In [2]:
column_list = list(df.columns.values)
for items in column_list:
    print(items)
rows = df.shape[0]

Symbol
Series
Date
Prev Close
Open Price
High Price
Low Price
Last Price
Close Price
Average Price
Total Traded Quantity
Turnover
No. of Trades
Deliverable Qty
% Dly Qt to Traded Qty


In [3]:
header = ["Date", "Open Price", "High Price", "Low Price", "Close Price"]
df.to_csv('output.csv', columns = header)

In [4]:
df = pd.read_csv('output.csv')
print(df[:10])

   Unnamed: 0         Date  Open Price  High Price  Low Price  Close Price
0           0  01-Jan-1997      790.00       810.0     790.00       790.10
1           1  02-Jan-1997      799.90       804.0     791.00       797.40
2           2  03-Jan-1997      791.05       799.0     791.05       796.00
3           3  06-Jan-1997      809.00       809.0     800.00       800.00
4           4  07-Jan-1997      790.05       803.0     790.05       801.70
5           5  08-Jan-1997      810.00       810.0     800.05       801.60
6           6  09-Jan-1997      800.00       809.5     800.00       809.45
7           7  10-Jan-1997      819.50       830.0     815.00       821.05
8           8  13-Jan-1997      829.90       832.9     827.10       832.90
9           9  14-Jan-1997      835.00       835.0     827.05       833.85


From the above data, we get the following
1. Upper Shadow
2. Body
3. Lower Shadow
4. Candle Color

For observations taken on each day.

## Upper Shadow
Size of the upper Shadow represents the sentiment of the buyers.

## Body
Body represents the intensity of the market sentiment.

## Lower Shadow
Size of the lower shadow represents the sentiment of the sellers.

## Candle Color
Shows whether the sentiment is gaining strength or losing.

## Assumption

The value of n to determine the universe of discourse is told to be 7. Here I have assumed that, the universe of discourse for any point is the average of 7 points prior to it. Not sure of this though. So we avoid the first 7 points of a given dataset?

In [5]:
df['Candle Color'] = 0
df['Upper Shadow'] = 0
df['Lower Shadow'] = 0

df['Body'] = abs(df['Open Price']-df['Close Price'])

for i in range(rows):
    if df['Open Price'][i]>df ['Close Price'][i]:
        df['Candle Color'][i] =  'Black'
        df['Upper Shadow'][i] = abs(df['Open Price'][i] - df['High Price'][i])
        df['Lower Shadow'][i] = abs(df['Close Price'][i] - df['Low Price'][i])
    else:
        df['Candle Color'][i] =  'White'
        df['Upper Shadow'][i] = abs(df['Close Price'][i] - df['High Price'][i])
        df['Lower Shadow'][i] = abs(df['Open Price'][i] - df['Low Price'][i])


In [6]:
df['universe_disc'] = 0
for i in range(rows-7):
    df['universe_disc'][i+7] = (sum(list(df['Body'][i:i+7])))/7

In [7]:
df['a'] = 0.15*df['universe_disc']
df['b'] = 0.30*df['universe_disc']
df['c'] = 0.45*df['universe_disc']
df['d'] = 0.60*df['universe_disc']
df['e'] = 0.75*df['universe_disc']

In [19]:
print(df[:100])

    Unnamed: 0         Date  Open Price  High Price  Low Price  Close Price  \
0            0  01-Jan-1997      790.00      810.00     790.00       790.10   
1            1  02-Jan-1997      799.90      804.00     791.00       797.40   
2            2  03-Jan-1997      791.05      799.00     791.05       796.00   
3            3  06-Jan-1997      809.00      809.00     800.00       800.00   
4            4  07-Jan-1997      790.05      803.00     790.05       801.70   
5            5  08-Jan-1997      810.00      810.00     800.05       801.60   
6            6  09-Jan-1997      800.00      809.50     800.00       809.45   
7            7  10-Jan-1997      819.50      830.00     815.00       821.05   
8            8  13-Jan-1997      829.90      832.90     827.10       832.90   
9            9  14-Jan-1997      835.00      835.00     827.05       833.85   
10          10  15-Jan-1997      858.50      870.00     858.00       869.90   
11          11  16-Jan-1997      885.00      885.00 

## Assumption

Couldn't figure out what exactly x meant, when they try fuzzifying. So took x as D<sub>i</sub><sup>OPCL</sup> as they have mentioned it can be either that or US<sub>i</sub> or LS<sub>i</sub>

In [9]:
df['mTNY'] = 0
df['mVS'] = 0
df['mSM'] = 0
df['mBG'] = 0
df['mVBG'] = 0


for i in range(rows):
    a = df['a'][i]
    b = df['b'][i]
    c = df['c'][i]
    d = df['d'][i]
    e = df['e'][i]
    x = df['Body'][i]
#     print(x, a, b, c, d, e)
    if x<=a:
        df['mTNY'][i] = 1
    elif x<b and x>a:
        df['mTNY'][i] = (b-x)/(b-a)
    else:
        df['mTNY'][i] = 0
        
        
    if x==b:
        df['mVS'][i] = 1
    elif x<b and x>a:
        df['mVS'][i] = (x-a)/(b-a)
    elif x>b and x<c:
        df['mVS'][i] = (c-x)/(c-b)
    else:
        df['mVS'][i] = 0
        
        
    if x==c:
        df['mSM'][i] = 1
    elif x<c and x>b:
        df['mSM'][i] = (x-b)/(c-b)
    elif x<d and x>c:
        df['mSM'][i] = (d-x)/(d-c)
    else:
        df['mSM'][i] = 0
        
        
    if x==d:
        df['mBG'][i] = 1
    elif x<d and x>c:
        df['mBG'][i] = (x-c)/(d-c)
    elif x<e and x>d:
        df['mBG'][i] = (e-x)/(e-d)
    else:
        df['mBG'][i] = 0
        
        
    if x>=e:
        df['mVBG'][i] = 1
    elif x<e and x>d:
        df['mVBG'][i] = (x-d)/(b-a)
    else:
        df['mVBG'][i] = 0

### 4.1.2
Fuzzification of the trend of closing values before and after a particular point p in the time series

In [10]:
df['Bias'] = 0
df['Magnitude'] = 0
for i in range(3, rows):
    df['Magnitude'][i] = abs(df['Close Price'][i] - df['Close Price'][i-3])
    if df['Close Price'][i]>=df ['Close Price'][i-3]:
        df['Bias'][i] = 'Positive'
    else:
        df['Bias'][i] = 'Negative'

## Assumption
We now make the Fuzzy(x). Here again x is taken as D<sub>i</sub><sup>OPCL</sup>

In [11]:
df['fuzzy'] = ''
for i in range(rows):
    max_val = max(df['mTNY'][i], df['mSM'][i], df['mVS'][i], df['mBG'][i], df['mVBG'][i])
    if df['mTNY'][i] == max_val:
        df['fuzzy'][i] = 'TNY'
    elif df['mVS'][i] == max_val:
        df['fuzzy'][i] = 'VS'
    elif df['mSM'][i] == max_val:
        df['fuzzy'][i] = 'SM'
    elif df['mBG'][i] == max_val:
        df['fuzzy'][i] = 'BG'
    else:
        df['fuzzy'][i] = 'VB'

In [20]:
df['fm'] = 0
for i in range(rows):
#     print(df['mTNY'][i], df['mSM'][i], df['mVS'][i], df['mBG'][i], df['mVBG'][i])
    if (df['Bias'][i] == "Positive" or df['Bias'][i]=='Negative') and df['fuzzy'][i] == 'TNY':
        df['fm'][i] = "Neutral"
    if df['Bias'][i]=="Positive" and df['fuzzy'][i]=='VS':
        df['fm'][i] = "Bullish Neutral"
    if df['Bias'][i]=="Negative" and df['fuzzy'][i]=='VS':
        df['fm'][i] = "Bearish Neutral"
    if df['Bias'][i]=="Positive" and df['fuzzy'][i]=='BG':
        df['fm'][i] = "Very Bullish"
    if df['Bias'][i]=="Negative" and df['fuzzy'][i]=='BG':
        df['fm'][i] = "Very Bearish"
    if df['Bias'][i]=="Positive" and df['fuzzy'][i]=='VB':
        df['fm'][i] = "Extremely Bullish"
    if df['Bias'][i]=="Negative" and df['fuzzy'][i]=='VB':
        df['fm'][i] = "Extremely Bearish"
# print(df)

In [21]:
df['md'] = 0
for i in range(rows):
    if df['fm'][i]=="Bearish Neutral" or df['fm'][i]=="Extremely Bearish":
        df['md'][i] = 'BR'
    if df['fm'][i]=="Bullish Neutral" or df['fm'][i]=="Very Bullish" or df['fm'][i]=="Extremely Bullish":
        df['md'][i] = 'BL'
    else:
        df['md'] = 'NT'


FM<sub>i</sub> represents the market momentum.


There are seven rules for determining this which have been performed using the IF conditions.

In [22]:
print(df[:100])

    Unnamed: 0         Date  Open Price  High Price  Low Price  Close Price  \
0            0  01-Jan-1997      790.00      810.00     790.00       790.10   
1            1  02-Jan-1997      799.90      804.00     791.00       797.40   
2            2  03-Jan-1997      791.05      799.00     791.05       796.00   
3            3  06-Jan-1997      809.00      809.00     800.00       800.00   
4            4  07-Jan-1997      790.05      803.00     790.05       801.70   
5            5  08-Jan-1997      810.00      810.00     800.05       801.60   
6            6  09-Jan-1997      800.00      809.50     800.00       809.45   
7            7  10-Jan-1997      819.50      830.00     815.00       821.05   
8            8  13-Jan-1997      829.90      832.90     827.10       832.90   
9            9  14-Jan-1997      835.00      835.00     827.05       833.85   
10          10  15-Jan-1997      858.50      870.00     858.00       869.90   
11          11  16-Jan-1997      885.00      885.00 